In [3]:
import pandas as pd

In [4]:
# Load natural gas data (provided by J.P. Morgan)
df = pd.read_csv("Nat_Gas (1).csv")
df['Dates'] = pd.to_datetime(df['Dates'])
df.set_index('Dates', inplace=True)


C:\Users\k8011\AppData\Local\Temp\ipykernel_12876\2958004267.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])


In [6]:
# -------------------------------
# Gas storage pricing model
# -------------------------------
def price_gas_storage_contract(injection_dates, withdrawal_dates, prices_df,
                               injection_rate, withdrawal_rate, max_storage, storage_cost):
    """
    Calculate the total value of a natural gas storage contract.
    """
    storage = 0
    total_cost = 0
    total_revenue = 0
    # Injection phase (buy gas)
    for date in injection_dates:
        date = pd.to_datetime(date)
        if date in prices_df.index:
            if storage + injection_rate <= max_storage:
                buy_price = prices_df.loc[date, 'Prices']
                total_cost += buy_price * injection_rate
                storage += injection_rate
            else:
                print(f"⚠️ Storage full on {date.date()}, cannot inject more gas.")
        else:
            print(f"⚠️ Price not available for {date.date()}")

    # Withdrawal phase (sell gas)
    for date in withdrawal_dates:
        date = pd.to_datetime(date)
        if date in prices_df.index:
            if storage - withdrawal_rate >= 0:
                sell_price = prices_df.loc[date, 'Prices']
                total_revenue += sell_price * withdrawal_rate
                storage -= withdrawal_rate
            else:
                print(f"⚠️ Not enough gas to withdraw on {date.date()}")
        else:
            print(f"⚠️ Price not available for {date.date()}")

    # Storage cost
    total_storage_days = len(injection_dates) + len(withdrawal_dates)
    total_storage_cost = total_storage_days * storage_cost

    # Final value of the contract
    contract_value = total_revenue - total_cost - total_storage_cost
    return contract_value


In [7]:
# -------------------------------
# Example test run
# -------------------------------
injection_dates = ['10/31/20', '11/30/20']  # Buying in early months
withdrawal_dates = ['2/28/21', '3/31/21']   # Selling later
injection_rate = 1e6        # 1 million MMBtu per injection
withdrawal_rate = 1e6       # 1 million MMBtu per withdrawal
max_storage = 2e6           # Max 2 million MMBtu storage capacity
storage_cost = 50000        # $50,000 per period (month)


In [8]:
# Calculate contract value
value = price_gas_storage_contract(
    injection_dates,
    withdrawal_dates,
    df,
    injection_rate,
    withdrawal_rate,
    max_storage,
    storage_cost
)


In [9]:
print("💰 Estimated Contract Value: $", round(value, 2))

💰 Estimated Contract Value: $ 1200000.0
